This notebook uses ipython-sql (pip install ipython-sql) to type SQL queries directly. Use %sql before a single-line query and %%sql before a multi-line query.

In [1]:
%load_ext sql

In [2]:
%sql postgresql://postgres@localhost/birth_db

u'Connected: postgres@birth_db'

In [18]:
# use SET search_path TO schema_name if schema is used
%sql reset search_path

Done.


[]

In [3]:
%%sql
SELECT * FROM birth_data_table
LIMIT 10
OFFSET 50

10 rows affected.


index,alcohol_use,anencephaly,attendant,birth_loc_type,birth_month,birth_state,birth_weight,birth_year,cigarette_use,cigarettes_per_day,cigarettes_trimester1,cigarettes_trimester2,cigarettes_trimester3,day,delivery_method,downs syndrome,drinks_per_week,father_age,father_race,gestation_weeks,infant_sex,mother_age,mother_birth_country,mother_birth_state,mother_education,mother_marital_status,mother_race,mother_state,population,pregnancy_weight,resident,revision,spina_bifida,table,timestamp,uses_tobacco,weight_gain
50,None,None,MD,None,Jan,None,5000.0,2012,None,None,None,None,None,Fri,Vaginal,None,None,None,None,42.0,F,20,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1325456800,None,30.0
51,None,None,MD,None,Jan,None,3000.0,2012,None,None,None,None,None,Fri,Vaginal,None,None,None,None,42.0,F,20,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1326371490,None,32.0
52,None,None,MD,None,Jan,None,3000.0,2012,None,None,None,None,None,Sun,Vaginal,None,None,None,None,38.0,F,21,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1327245443,None,24.0
53,None,None,MD,None,Feb,None,4000.0,2012,None,None,None,None,None,Wed,Cesarean,None,None,None,None,39.0,M,35,None,None,None,Yes,White,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1328148996,None,30.0
54,None,None,MD,None,Feb,None,3000.0,2012,None,None,None,None,None,Wed,Vaginal,None,None,None,None,38.0,F,29,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1328933795,None,34.0
55,None,None,MD,None,Feb,None,2000.0,2012,None,None,None,None,None,Sat,Vaginal,None,None,None,None,31.0,M,27,None,None,None,No,Black,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1329753116,None,25.0
56,None,None,MD,None,Mar,None,4000.0,2012,None,None,None,None,None,Tu,Vaginal,None,None,None,None,40.0,F,18,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1330659032,None,50.0
57,None,None,MD,None,Mar,None,4000.0,2012,None,None,None,None,None,Sun,Vaginal,None,None,None,None,38.0,F,29,None,None,None,Yes,Black,None,None,None,Resident,S,None,births12.txt,1331541002,None,50.0
58,None,None,MD,None,Mar,None,4000.0,2012,None,None,None,None,None,Mon,Vaginal,None,None,None,None,39.0,F,29,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1332469424,None,31.0
59,None,None,MD,None,Apr,None,4000.0,2012,None,None,None,None,None,Sat,Vaginal,None,None,None,None,39.0,M,30,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1333389308,None,3.0


What is the most common day for a delivery?

Let's first create a view containing the days of the week and the count for each day

In [19]:
%%sql 
CREATE VIEW day_counts AS
SELECT day, COUNT(*) AS count
FROM birth_data_table
GROUP BY day

(psycopg2.ProgrammingError) relation "day_counts" already exists
 [SQL: 'CREATE VIEW day_counts AS\nSELECT day, COUNT(*) AS count\nFROM birth_data_table\nGROUP BY day']


In [44]:
%sql SELECT * FROM day_counts

7 rows affected.


day,count
Tu,449
Sat,307
Mon,403
Wed,415
Sun,281
Th,432
Fri,429


We can view this table ordered by count.

In [18]:
%%sql 
SELECT * 
FROM day_counts 
ORDER BY count DESC  -- return in descending order

7 rows affected.


day,count
Tu,449
Th,432
Fri,429
Wed,415
Mon,403
Sat,307
Sun,281


For a solution with one query and one sub-query, see below. Note that this solution creates a temporary table instead of a view.

In [17]:
%%sql
SELECT day
FROM (SELECT day, COUNT(*) AS count   -- the parenthesis enclose a subquery
      FROM birth_data_table
      GROUP BY day
     ) AS day_counts       -- name the temporary table
WHERE count = (SELECT MAX(count) FROM day_counts)

1 rows affected.


day
Tu


Now let's get counts by attendant. First we'll see what unique values are in the table.

In [20]:
%sql SELECT DISTINCT attendant FROM birth_data_table

5 rows affected.


attendant
None
Certified Nurse or Midwife
MD
Osteopathy
Other Midwife


In [21]:
%%sql
SELECT attendant, COUNT(*) as count
FROM birth_data_table
GROUP BY attendant

5 rows affected.


attendant,count
None,16
Certified Nurse or Midwife,248
MD,2285
Osteopathy,154
Other Midwife,13


Hmm, I wonder if there is any relation between attendant and day of the week for a delivery. Let's make a pivot table, where one column is attendant and the other columns are the days of the week, containing counts for each attendant for each day.

In [25]:
%%sql
SELECT attendant, day, COUNT(*) as count
FROM birth_data_table
GROUP BY attendant, day

34 rows affected.


attendant,day,count
Other Midwife,Tu,2
None,Sun,2
Other Midwife,Fri,1
MD,Tu,381
Osteopathy,Tu,23
Certified Nurse or Midwife,Sat,38
Other Midwife,Sat,3
Certified Nurse or Midwife,Fri,33
Certified Nurse or Midwife,Tu,39
None,Mon,2


That query gave us a long-form table, but we want a short-form table, where row-level information is converted into columns.

CROSSTAB (short for cross tabulation) is the PostgreSQL version of PIVOT, demonstrated below. First, load the extension - this needs to be done once per database. Note: you need to be logged in as a superuser to do this.

In [44]:
%sql CREATE EXTENSION tablefunc

Done.


[]

First, find the distinct values for the column of interest. We will hard-code these as the new columns.

In [45]:
%sql SELECT DISTINCT day FROM birth_data_table

7 rows affected.


day
Tu
Sat
Mon
Wed
Sun
Th
Fri


In [54]:
%%sql
SELECT *
FROM crosstab(
    'SELECT attendant, day, COUNT(*) as ct FROM birth_data_table GROUP BY attendant, day ORDER BY 1,2'
      -- below, we list all the unique values in day
      -- these will become the columns
      -- hard-coding them ensures that the order matches what we specify later
,$$VALUES ('Sun'::text), ('Mon'::text), ('Tu'::text), ('Wed'::text), ('Th'::text), ('Fri'::text), ('Sat'::text)$$
)
AS ct (
  -- first column has each unique value for the rows
"Attendant" text,
  -- now we list the columns
"Sun" text, "Mon" text, "Tu" text, "Wed" text, "Th" text, "Fri" text, "Sat" text
);

5 rows affected.


Attendant,Sun,Mon,Tu,Wed,Th,Fri,Sat
Certified Nurse or Midwife,30,39,39,35,34,33,38
MD,233,333,381,354,367,362,255
Osteopathy,13,26,23,23,29,30,10
Other Midwife,3,3,2,1,None,1,3
None,2,2,4,2,2,3,1


For more details, see: https://stackoverflow.com/questions/3002499/postgresql-crosstab-query/11751905#11751905

Testing date calculations

In [18]:
%%sql
SELECT *, (EXTRACT(year FROM current_date) - birth_year) AS age
FROM birth_data_table
LIMIT 5

5 rows affected.


index,alcohol_use,anencephaly,attendant,birth_loc_type,birth_month,birth_state,birth_weight,birth_year,cigarette_use,cigarettes_per_day,cigarettes_trimester1,cigarettes_trimester2,cigarettes_trimester3,day,delivery_method,downs syndrome,drinks_per_week,father_age,father_race,gestation_weeks,infant_sex,mother_age,mother_birth_country,mother_birth_state,mother_education,mother_marital_status,mother_race,mother_state,population,pregnancy_weight,resident,revision,spina_bifida,table,timestamp,uses_tobacco,weight_gain,age
0,None,None,MD,None,Jan,None,4500.0,2012,None,None,None,None,None,Fri,Vaginal,None,None,None,None,40.0,M,32,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1325405431,None,40.0,5.0
1,None,None,MD,None,Jan,None,4500.0,2012,None,None,None,None,None,Wed,Cesarean,None,None,None,None,39.0,F,28,None,None,None,Yes,White,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1325882986,None,49.0,5.0
2,None,None,MD,None,Jan,None,2500.0,2012,None,None,None,None,None,Sat,Cesarean,None,None,None,None,36.0,M,28,None,None,None,No,Black,None,None,None,Resident,S,None,births12.txt,1326367089,None,30.0,5.0
3,None,None,MD,None,Jan,None,3000.0,2012,None,None,None,None,None,Sun,Vaginal,None,None,None,None,40.0,M,28,None,None,None,No,Black,None,None,None,Resident,S,None,births12.txt,1326838810,None,15.0,5.0
4,None,None,MD,None,Jan,None,4000.0,2012,None,None,None,None,None,Th,Vaginal,None,None,None,None,39.0,F,22,None,None,None,No,American Indian,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1327339729,None,48.0,5.0


In [14]:
%sql SELECT EXTRACT(year FROM CURRENT_DATE)

1 rows affected.


date_part
2017.0


In [28]:
%%sql
WITH age_table AS 
(SELECT birth_month, birth_year, (EXTRACT(year FROM current_date) - birth_year) AS age 
FROM birth_data_table)
SELECT birth_month, AVG(age), MIN(age), MAX(age)
FROM age_table
GROUP BY birth_month

1 rows affected.


avg,min,max
5.0,5.0,5.0
